In [1]:
import pandas as pd 
import re
from splinter import Browser
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pymongo

In [2]:
full_df = pd.read_csv('../Resources/lyric_df.csv')
lyrics_df = pd.read_csv('../Resources/english_lyrics.csv')


In [3]:
full_df

,Unnamed: 0,track,album,artist,popularity,featured,release_date,track_id,danceability,energy,...,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics
0,0,BIBI Vengeance,Lowlife Princess: Noir,BIBI,76,BIBI,2022-11-17,0ra3bPUOj2YnY4FJHXtgHZ,0.442,0.748,...,-4.718,0,0.0895,0.000128,0.0642,0.408,99.981,165135,4,"[비비 ""나쁜년"" 가사][Verse 1]들어오라 해눈을 깔어 왜비밀로 할게네 부정과..."
1,1,KAZINO,KAZINO,BIBI,68,BIBI,2020-04-29,245onUPHGD1DYuiacxTuW0,0.625,0.686,...,-3.715,0,0.0414,0.000000,0.1100,0.263,133.041,185947,4,"[비비 ""사장님 도박은 재미로 하셔야 합니다"" 가사][Verse 1]Dice and..."
2,2,Animal Farm,Animal Farm,BIBI,67,BIBI,2022-09-27,3gAXVQOR0h7Ks71lsr5MFZ,0.617,0.570,...,-4.280,0,0.0308,0.000006,0.5500,0.375,107.971,203946,3,"[비비 ""가면무도회"" 가사][Verse 1]죄악의 단어위선의 가면넘쳐흐르는 무도회장..."
3,3,BAD SAD AND MAD,Life is a Bi…,BIBI,66,BIBI,2021-04-28,0zPbUxvYRlCGwruakkXPLI,0.746,0.558,...,-7.470,0,0.0880,0.000190,0.0904,0.638,90.013,154988,4,"[비비 ""BAD SAD AND MAD"" 가사][Refrain]You blued my..."
4,4,"Very, Slowly",Twenty-Five Twenty-One OST Part 3,BIBI,62,BIBI,2022-02-20,7GkHIsnziYgk6j1lx2TK6H,0.528,0.423,...,-7.381,1,0.0264,0.001480,0.1670,0.249,135.917,214852,4,missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,453,HANDSOME,BOY OF THE YEAR,Warren Hue,45,Warren Hue,2022-07-29,58z1hY9hrwliXRIvQe8UOX,0.762,0.686,...,-6.113,0,0.1760,0.000000,0.0846,0.385,89.992,126962,4,[Verse 1: Warren Hue](I wanna be like)I've bee...
454,454,W (with yvngxchris),BOY OF THE YEAR,Warren Hue,39,yvngxchris,2022-07-29,4U2BfAHYdfDG25XL9eG0BO,0.719,0.504,...,-6.449,1,0.4220,0.000000,0.2150,0.493,164.926,151446,4,missing
455,455,DEMOSTAR BEENLIT,BOY OF THE YEAR,Warren Hue,39,Warren Hue,2022-07-29,647TFpvzp7uPAAvOfetag9,0.779,0.871,...,-5.406,1,0.2900,0.000003,0.0995,0.532,162.055,126043,4,"[Intro]Uh, uh, ayy, ya-yaUh, uh, turn-turn, ya..."
456,456,RUNAWAY W ME,BOY OF THE YEAR,Warren Hue,42,Warren Hue,2022-07-29,3EXg5PZgK1dwZyzqLp9qES,0.651,0.677,...,-7.769,1,0.1380,0.000000,0.2470,0.324,129.923,181840,4,"[Chorus]Won't you, run away with me this eveni..."


In [4]:
full_df = full_df.drop(['Unnamed: 0'], axis=1)

In [5]:
full_df

,track,album,artist,popularity,featured,release_date,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics
0,BIBI Vengeance,Lowlife Princess: Noir,BIBI,76,BIBI,2022-11-17,0ra3bPUOj2YnY4FJHXtgHZ,0.442,0.748,11,-4.718,0,0.0895,0.000128,0.0642,0.408,99.981,165135,4,"[비비 ""나쁜년"" 가사][Verse 1]들어오라 해눈을 깔어 왜비밀로 할게네 부정과..."
1,KAZINO,KAZINO,BIBI,68,BIBI,2020-04-29,245onUPHGD1DYuiacxTuW0,0.625,0.686,6,-3.715,0,0.0414,0.000000,0.1100,0.263,133.041,185947,4,"[비비 ""사장님 도박은 재미로 하셔야 합니다"" 가사][Verse 1]Dice and..."
2,Animal Farm,Animal Farm,BIBI,67,BIBI,2022-09-27,3gAXVQOR0h7Ks71lsr5MFZ,0.617,0.570,5,-4.280,0,0.0308,0.000006,0.5500,0.375,107.971,203946,3,"[비비 ""가면무도회"" 가사][Verse 1]죄악의 단어위선의 가면넘쳐흐르는 무도회장..."
3,BAD SAD AND MAD,Life is a Bi…,BIBI,66,BIBI,2021-04-28,0zPbUxvYRlCGwruakkXPLI,0.746,0.558,0,-7.470,0,0.0880,0.000190,0.0904,0.638,90.013,154988,4,"[비비 ""BAD SAD AND MAD"" 가사][Refrain]You blued my..."
4,"Very, Slowly",Twenty-Five Twenty-One OST Part 3,BIBI,62,BIBI,2022-02-20,7GkHIsnziYgk6j1lx2TK6H,0.528,0.423,8,-7.381,1,0.0264,0.001480,0.1670,0.249,135.917,214852,4,missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,HANDSOME,BOY OF THE YEAR,Warren Hue,45,Warren Hue,2022-07-29,58z1hY9hrwliXRIvQe8UOX,0.762,0.686,10,-6.113,0,0.1760,0.000000,0.0846,0.385,89.992,126962,4,[Verse 1: Warren Hue](I wanna be like)I've bee...
454,W (with yvngxchris),BOY OF THE YEAR,Warren Hue,39,yvngxchris,2022-07-29,4U2BfAHYdfDG25XL9eG0BO,0.719,0.504,6,-6.449,1,0.4220,0.000000,0.2150,0.493,164.926,151446,4,missing
455,DEMOSTAR BEENLIT,BOY OF THE YEAR,Warren Hue,39,Warren Hue,2022-07-29,647TFpvzp7uPAAvOfetag9,0.779,0.871,1,-5.406,1,0.2900,0.000003,0.0995,0.532,162.055,126043,4,"[Intro]Uh, uh, ayy, ya-yaUh, uh, turn-turn, ya..."
456,RUNAWAY W ME,BOY OF THE YEAR,Warren Hue,42,Warren Hue,2022-07-29,3EXg5PZgK1dwZyzqLp9qES,0.651,0.677,4,-7.769,1,0.1380,0.000000,0.2470,0.324,129.923,181840,4,"[Chorus]Won't you, run away with me this eveni..."


# Clean/Process Lyrics

In [6]:
def remove_non_english(sentence):
    # Split the sentence into words
    words = sentence.split()
    
    # Use a regular expression to remove all non-alphabetic characters and keep only English words
    words = [re.sub(r'[^a-zA-Z]', ' ', word) for word in words]
    
    # Join the words back into a sentence
    sentence = ' '.join(words)
    
    return sentence

In [7]:
def remove_song_structure(lyrics):
    # Use a regular expression to remove the specific song structure elements
    lyrics = re.sub(r'\[(Pre-Chorus|Verse 1|Chorus|Intro|Verse|Verse 2|Outro|Bridge)\]', ' ', lyrics)
    
    return lyrics

In [8]:
def remove_extra_spaces(text):
    # Use a regular expression to remove multiple consecutive spaces
    text = re.sub(r' {2,}', ' ', text)
    
    return text

In [9]:
def remove_verse(lyrics):
    # Use a regular expression to remove the word "verse"
    lyrics = re.sub(r'Verse', '', lyrics, flags=re.IGNORECASE)
    
    return lyrics

In [10]:
full_df['filtered_lyrics'] = full_df['lyrics'].apply(remove_song_structure).apply(remove_non_english).apply(remove_extra_spaces).apply(remove_verse)

In [11]:
full_df

,track,album,artist,popularity,featured,release_date,track_id,danceability,energy,key,...,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics,filtered_lyrics
0,BIBI Vengeance,Lowlife Princess: Noir,BIBI,76,BIBI,2022-11-17,0ra3bPUOj2YnY4FJHXtgHZ,0.442,0.748,11,...,0,0.0895,0.000128,0.0642,0.408,99.981,165135,4,"[비비 ""나쁜년"" 가사][Verse 1]들어오라 해눈을 깔어 왜비밀로 할게네 부정과...",Watch me danceWatch me sing Watch me danceWat...
1,KAZINO,KAZINO,BIBI,68,BIBI,2020-04-29,245onUPHGD1DYuiacxTuW0,0.625,0.686,6,...,0,0.0414,0.000000,0.1100,0.263,133.041,185947,4,"[비비 ""사장님 도박은 재미로 하셔야 합니다"" 가사][Verse 1]Dice and...",Dice and Ice cube Home girls bitches Spin it ...
2,Animal Farm,Animal Farm,BIBI,67,BIBI,2022-09-27,3gAXVQOR0h7Ks71lsr5MFZ,0.617,0.570,5,...,0,0.0308,0.000006,0.5500,0.375,107.971,203946,3,"[비비 ""가면무도회"" 가사][Verse 1]죄악의 단어위선의 가면넘쳐흐르는 무도회장...",champagne It might be pretty it might be beau...
3,BAD SAD AND MAD,Life is a Bi…,BIBI,66,BIBI,2021-04-28,0zPbUxvYRlCGwruakkXPLI,0.746,0.558,0,...,0,0.0880,0.000190,0.0904,0.638,90.013,154988,4,"[비비 ""BAD SAD AND MAD"" 가사][Refrain]You blued my...",BAD SAD AND MAD Refrain You blued my bruisePi...
4,"Very, Slowly",Twenty-Five Twenty-One OST Part 3,BIBI,62,BIBI,2022-02-20,7GkHIsnziYgk6j1lx2TK6H,0.528,0.423,8,...,1,0.0264,0.001480,0.1670,0.249,135.917,214852,4,missing,missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,HANDSOME,BOY OF THE YEAR,Warren Hue,45,Warren Hue,2022-07-29,58z1hY9hrwliXRIvQe8UOX,0.762,0.686,10,...,0,0.1760,0.000000,0.0846,0.385,89.992,126962,4,[Verse 1: Warren Hue](I wanna be like)I've bee...,Warren Hue I wanna be like I ve been stuck i...
454,W (with yvngxchris),BOY OF THE YEAR,Warren Hue,39,yvngxchris,2022-07-29,4U2BfAHYdfDG25XL9eG0BO,0.719,0.504,6,...,1,0.4220,0.000000,0.2150,0.493,164.926,151446,4,missing,missing
455,DEMOSTAR BEENLIT,BOY OF THE YEAR,Warren Hue,39,Warren Hue,2022-07-29,647TFpvzp7uPAAvOfetag9,0.779,0.871,1,...,1,0.2900,0.000003,0.0995,0.532,162.055,126043,4,"[Intro]Uh, uh, ayy, ya-yaUh, uh, turn-turn, ya...",Uh uh ayy ya yaUh uh turn turn ya yaT turn ya ...
456,RUNAWAY W ME,BOY OF THE YEAR,Warren Hue,42,Warren Hue,2022-07-29,3EXg5PZgK1dwZyzqLp9qES,0.651,0.677,4,...,1,0.1380,0.000000,0.2470,0.324,129.923,181840,4,"[Chorus]Won't you, run away with me this eveni...",Won t you run away with me this evening Oh tru...


In [58]:
test_df = pd.read_csv('../Resources/sample_df.csv', usecols=["track", "artist"])

# Load a test dataframe for trial

In [59]:
test_df

,track,artist
0,BIBI Vengeance,BIBI
1,KAZINO,BIBI
2,Animal Farm,BIBI
3,BAD SAD AND MAD,BIBI
4,Restless,BIBI
...,...,...
276,THIRTYNINE,Warren Hue
277,HANDSOME,Warren Hue
278,DEMOSTAR BEENLIT,Warren Hue
279,RUNAWAY W ME,Warren Hue


# Create a scrape genre function

In [84]:
#Optimal function to scrape genre tag from genius
def scrape_genre(artist,song):

    url = f'https://genius.com/{artist}-{song}-lyrics'
    
    # Make a request to the website and parse the HTML response
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    div_class = soup.find_all('div', class_="SongTags__Container-xixwg3-1 bZsZHM")
    
    if div_class:
        genre_tags = []
        for r in div_class:
            anchor = r.find_all('a')
            for a in anchor:
                genre_tags.append(a.text)
        return genre_tags
    else:
        return []

In [61]:
data = []

for index, row in test_df.iterrows():

    artist = row['artist'].replace(' ','-') 
    song = row['track'].replace(' ','-')

    genre_tags = scrape_genre(artist,song)
    
    # Append the genre tags to the list
    data.append(genre_tags)

# Add the genre_tags_list to the dataframe as a new column 'genre_tags'
test_df['genre'] = data

In [62]:
test_df

,track,artist,genre
0,BIBI Vengeance,BIBI,"[R&B, K-R&B, Reggaetón, South Korea, Genius Ko..."
1,KAZINO,BIBI,"[R&B, South Korea, Trap, Korean]"
2,Animal Farm,BIBI,"[R&B, Korean, K-R&B, K-Solo, South Korea, Geni..."
3,BAD SAD AND MAD,BIBI,"[R&B, K-R&B, Korean, K-Solo, South Korea, Geni..."
4,Restless,BIBI,"[R&B, South Korea, K-R&B, Korean]"
...,...,...,...
276,THIRTYNINE,Warren Hue,"[Rap, Indonesia]"
277,HANDSOME,Warren Hue,"[Rap, Indonesia]"
278,DEMOSTAR BEENLIT,Warren Hue,[Rap]
279,RUNAWAY W ME,Warren Hue,"[Pop, Indonesia]"


In [87]:
data = []

for index, row in full_df.iterrows():

    artist = row['artist'].replace(' ','-') 
    song = row['track'].replace(' ','-')

    genre_tags = scrape_genre(artist,song)
    
    # Append the genre tags to the list
    data.append(genre_tags)

# Add the genre_tags_list to the dataframe as a new column 'genre_tags'
full_df['genre'] = data

In [89]:
full_df

,track,album,artist,popularity,featured,release_date,track_id,danceability,energy,key,...,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics,filtered_lyrics,genre
0,BIBI Vengeance,Lowlife Princess: Noir,BIBI,76,BIBI,2022-11-17,0ra3bPUOj2YnY4FJHXtgHZ,0.442,0.748,11,...,0.0895,0.000128,0.0642,0.408,99.981,165135,4,"[비비 ""나쁜년"" 가사][Verse 1]들어오라 해눈을 깔어 왜비밀로 할게네 부정과...",Watch me danceWatch me sing Watch me danceWat...,"[R&B, K-R&B, Reggaetón, South Korea, Genius Ko..."
1,KAZINO,KAZINO,BIBI,68,BIBI,2020-04-29,245onUPHGD1DYuiacxTuW0,0.625,0.686,6,...,0.0414,0.000000,0.1100,0.263,133.041,185947,4,"[비비 ""사장님 도박은 재미로 하셔야 합니다"" 가사][Verse 1]Dice and...",Dice and Ice cube Home girls bitches Spin it ...,"[R&B, South Korea, Trap, Korean]"
2,Animal Farm,Animal Farm,BIBI,67,BIBI,2022-09-27,3gAXVQOR0h7Ks71lsr5MFZ,0.617,0.570,5,...,0.0308,0.000006,0.5500,0.375,107.971,203946,3,"[비비 ""가면무도회"" 가사][Verse 1]죄악의 단어위선의 가면넘쳐흐르는 무도회장...",champagne It might be pretty it might be beau...,"[R&B, Korean, K-R&B, K-Solo, South Korea, Geni..."
3,BAD SAD AND MAD,Life is a Bi…,BIBI,66,BIBI,2021-04-28,0zPbUxvYRlCGwruakkXPLI,0.746,0.558,0,...,0.0880,0.000190,0.0904,0.638,90.013,154988,4,"[비비 ""BAD SAD AND MAD"" 가사][Refrain]You blued my...",BAD SAD AND MAD Refrain You blued my bruisePi...,"[R&B, K-R&B, Korean, K-Solo, South Korea, Geni..."
4,"Very, Slowly",Twenty-Five Twenty-One OST Part 3,BIBI,62,BIBI,2022-02-20,7GkHIsnziYgk6j1lx2TK6H,0.528,0.423,8,...,0.0264,0.001480,0.1670,0.249,135.917,214852,4,missing,missing,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,HANDSOME,BOY OF THE YEAR,Warren Hue,45,Warren Hue,2022-07-29,58z1hY9hrwliXRIvQe8UOX,0.762,0.686,10,...,0.1760,0.000000,0.0846,0.385,89.992,126962,4,[Verse 1: Warren Hue](I wanna be like)I've bee...,Warren Hue I wanna be like I ve been stuck i...,"[Rap, Indonesia]"
454,W (with yvngxchris),BOY OF THE YEAR,Warren Hue,39,yvngxchris,2022-07-29,4U2BfAHYdfDG25XL9eG0BO,0.719,0.504,6,...,0.4220,0.000000,0.2150,0.493,164.926,151446,4,missing,missing,[]
455,DEMOSTAR BEENLIT,BOY OF THE YEAR,Warren Hue,39,Warren Hue,2022-07-29,647TFpvzp7uPAAvOfetag9,0.779,0.871,1,...,0.2900,0.000003,0.0995,0.532,162.055,126043,4,"[Intro]Uh, uh, ayy, ya-yaUh, uh, turn-turn, ya...",Uh uh ayy ya yaUh uh turn turn ya yaT turn ya ...,[Rap]
456,RUNAWAY W ME,BOY OF THE YEAR,Warren Hue,42,Warren Hue,2022-07-29,3EXg5PZgK1dwZyzqLp9qES,0.651,0.677,4,...,0.1380,0.000000,0.2470,0.324,129.923,181840,4,"[Chorus]Won't you, run away with me this eveni...",Won t you run away with me this evening Oh tru...,"[Pop, Indonesia]"


In [92]:
full_df.to_csv('../Test-Data/pre-final-88-genre.csv', sep=',')

In [63]:
# test_df.to_csv('../Resources/trial_run_genre.csv', sep=',')